In [1]:
import numpy as np
from skimage.util.shape import view_as_windows
from numpy.lib.stride_tricks import as_strided

##### Input Layer

In [2]:
input_data_list = [[-1, 1, 1, 1,-1],
                   [-1, 1,-1, 1,-1],
                   [-1, 1, 1, 1,-1],
                   [-1,-1,-1, 1,-1],
                   [-1,-1,-1, 1,-1],
                   [-1,-1, 1,-1,-1],
                   [-1, 1,-1,-1,-1]]

# Convert list to input data matrix
input_data_matrix = np.asarray(a=input_data_list, dtype=np.float32)

print(input_data_matrix.shape)

(7, 5)


##### Convolution Layer(convolution operation)

In [3]:
box_filter_list = [[1, 1, 1],
                   [1,-1, 1],
                   [1, 1, 1]]

vertical_line_filter_list = [[-1, 1,-1],
                             [-1, 1,-1],
                             [-1, 1,-1]]

diagonal_line_filter_list = [[-1,-1, 1],
                             [-1, 1,-1],
                             [ 1,-1,-1]]

In [4]:
# Convert all filter lists into matrices
box_filter_matrix = np.asarray(box_filter_list, dtype=np.float32)
vertical_line_filter_matrix = np.asarray(vertical_line_filter_list, dtype=np.float32)
diagonal_line_filter_matrix = np.asarray(diagonal_line_filter_list, dtype=np.float32)

print(f"box_filter_matrix's shape : {box_filter_matrix.shape}")
print(f"vertical_line_filter_matrix's shape : {vertical_line_filter_matrix.shape}")
print(f"diagonal_line_filter_matrix's shape : {diagonal_line_filter_matrix.shape}")

box_filter_matrix's shape : (3, 3)
vertical_line_filter_matrix's shape : (3, 3)
diagonal_line_filter_matrix's shape : (3, 3)


In [5]:
# Extract each window from input matrix by stride operation
def strided4D_v2(input_matrix, kernel_matrix, stride):
    return view_as_windows(input_matrix, kernel_matrix.shape, step=stride)

In [6]:
# Calculate shape of the feature map (output matrix from convolution layer)   # (5, 3, 3, 3) : feature map's shape is (5, 3) ==> (5, 3, 3, 3)
featureMap_row = strided4D_v2(input_data_matrix, box_filter_matrix, 1).shape[0]  # 5
featureMap_col = strided4D_v2(input_data_matrix, box_filter_matrix, 1).shape[1]  # 3

print(f"featureMap_row : {featureMap_row},   featureMap_col : {featureMap_col}")

featureMap_row : 5,   featureMap_col : 3


In [7]:
# Calculate featuremap matrix for box filter
def conv2d(input_matrix, kernel_matrix):
    # Create blank featuremap matrix for stride 1
    featureMap_output = np.zeros((featureMap_row, featureMap_col))
    
    for row in range(featureMap_row):
        for col in range(featureMap_col):
            window = strided4D_v2(input_matrix, kernel_matrix, 1)[row][col]
            featureMap_output[row, col] = np.sum(np.multiply(kernel_matrix, window))
            
            # To format floats in numpy array
            np.set_printoptions(precision=2)
            
            # Take average with divided by 9(total number of elements in filter matrix)
            total_num_of_elements_in_filter_matrix = kernel_matrix.shape[0] * kernel_matrix.shape[1]
            
    return featureMap_output / total_num_of_elements_in_filter_matrix

##### Box Filter Operation

In [8]:
featureMap_box = conv2d(input_data_matrix, box_filter_matrix)
print(featureMap_box)

[[-0.11  1.   -0.11]
 [-0.56  0.11 -0.33]
 [-0.33  0.33 -0.33]
 [-0.56 -0.11 -0.56]
 [-0.33 -0.56 -0.33]]


##### Vertical Filter Operation

In [9]:
featureMap_vertical = conv2d(input_data_matrix, vertical_line_filter_matrix)
print(featureMap_vertical)

[[ 0.56 -0.56  0.56]
 [ 0.56 -0.56  0.78]
 [ 0.33 -0.33  0.78]
 [ 0.11  0.11  0.56]
 [ 0.33  0.11  0.33]]


##### Diagonal Filter Operation

In [10]:
featureMap_diagonal = conv2d(input_data_matrix, diagonal_line_filter_matrix)
print(featureMap_diagonal)

[[ 0.11 -0.56  0.11]
 [ 0.11 -0.11 -0.11]
 [ 0.33 -0.33 -0.11]
 [ 0.11  0.11  0.56]
 [-0.11  1.   -0.11]]


##### Box ReLU Operation

In [49]:
featureMap_box_relu = np.maximum(featureMap_box, 0)
featureMap_vertical_relu = np.maximum(featureMap_vertical, 0)
featureMap_diagonal_relu = np.maximum(featureMap_diagonal, 0)

# np.set_printoptions(suppress=True, linewidth=300)
np.set_printoptions(formatter={'float_kind':'{:.2f}'.format}, linewidth=300)
all_kinds_relu = np.concatenate((featureMap_box_relu, np.empty((5,1)), featureMap_vertical_relu, np.empty((5,1)), featureMap_diagonal_relu), axis=1)
print(all_kinds_relu)


[[0.00 1.00 0.00 0.00 0.56 0.00 0.56 0.00 0.11 0.00 0.11]
 [0.00 0.11 0.00 0.00 0.56 0.00 0.78 0.00 0.11 0.00 0.00]
 [0.00 0.33 0.00 0.00 0.33 0.00 0.78 0.00 0.33 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.11 0.11 0.56 0.00 0.11 0.11 0.56]
 [0.00 0.00 0.00 0.00 0.33 0.11 0.33 0.00 0.00 1.00 0.00]]


##### Pooling Layer